# Generating new examples

This notebook's purpose is to generate new examples of *dysfunctional matrices* - matrices for which the cube algorithm gives different result than the LLL algorithm. For now, this code generates only square matrices. It returns a json file containing the *dysfunctional* matrices and their core parameters.

The generating function takes in parameters above.

`dimension`: the shape of the matrix

`perimeter`: defines an interval `[- perimeter, perimeter]` for the integers inside the matrix

`sensitivity`: lol i forgot

`jsonfilename`: name of the ouput file

## Output format

Jsonfile containing a `list` of `dictionaries`, each dictionary representing single case of a *dysfunctional matrix*. The `dict` includes the following: 

`"B"` the original faulty matrix `B`

`"G"` it's corresponding Gram matrix

`"lincomb_LLL"` linear combination corresponding to the shortest vector according to the LLL algorithm

`"lincomb_cube"` linear combination corresponding to the shortest vector according to the cube algorithm

`"sv_LLL"` shortest vector according to LLL 

`"sv_cube"` shortest vector according to cube

`"lincomb_diff"` difference between the two linear combination (their matrices subtracted)

`"Diff impact"` detailed matrix multiplication of the difference matrix


## Code and commentary

### Utilitary math tools

These functions are uniteresting tools needed for the basic computations. Functions such as *finding the shortest vectors of a basis matrix*, *creating a random linearly independent basis matrix*, *computing Gram matrix of a given matrix* or *converting between data types*

In [92]:
# UNINTERESTING TOOLS
from random import randint, seed




def shortestVector(matrix): 
    """
    :returns: 
        - n - norm of the shortest vector
        - v - the shortest vector (vector)
        - i - index of the SV
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0][1]

def randomMatrix(dimension, per) -> matrix:
    '''
    Returns a random square matrix with full rank.
    INPUT:
    dimension: dimension of the random matrix
    per: perimeter of the components
    '''
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]

### Interesting math tools. 

Following functions actually have some thoughts behind them. 


`abnormality_data`
Given a matrix $B$ and it's shortest vector given by the LLL algorithm, returns __a dictionary__ with data concerting the SV given by the cube algorithm, if it's far enough from the LLL solution. 

`find_real_minimum` Given a matrix $B$ and it's shortest vector given by the LLL algorithm, returns the __linear combination of the SV given by the cube algorithm__.




### Formatting tools

These functions provide transfering the data to the output jsonfile as described above.

In [93]:
### output formatting
import json
import numpy as np

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            elif isinstance(value, (int, float, sage.rings.real_mpfr.RealNumber)):
                dic[key] = float(value)  # Directly use the float value
            else:
                dic[key] = vector_to_list(value)
    return output_data


def into_json(data, filename="data.json"):
    with open(filename, 'w') as f:
        json.dump(data, f, default=custom_serializer)

def from_json(filename):
    out_file = open(filename)
    return json.load(out_file)

def print_listdict(list) -> None:
    """
    :param list: list of dictionaries
    """
    for dictionary in list:
        for pair in dictionary.items():
            print(pair[1], ": ", pair[0])
        print()
    
def into_dict(B, lcLLL: vector, rowindex: int) -> dict:
    result = {}
    G = gram_matrix(B)
    lcCube = find_real_minimum(G, rowindex, lcLLL[rowindex])
    result["B"] = B
    result["G"] = G
    result["lincomb_LLL"] = lcLLL
    result["lincomb_cube"] = lcCube
    result["sv_LLL"] = lcLLL * B
    result["sv_cube"] = lcCube * B
    result["lincomb_diff"] = lcLLL - vector(lcCube)
    result["LLL.norm"] = vector(lcLLL*B).norm().n(digits=5)
    result["cube.norm"] = (vector(lcCube)*B).norm().n(digits=5)
    return result
    

### Main function

This cell actually generates the new examples and checks wheter a matrix is *dysfunctional*. 

`generate_new_examples`
The main function. Based on input parameters above, generates *some* number of *dysfunctional* matrices, computes their invariants such as Gram matrix, LLL/cube linear combinations etc. and creates a json file with this information (specified above).

`is_dysfunctional`
Given a matrix $B$, checks whether its dysfunctional and if so, returns the data describing the case.


In [94]:
# MAIN 
def generate_new_examples(iterations, dimension, perimeter, printing = False, functioning = True) -> None:
    '''
    generates >iterations< of >dis/functioning< matrices and saves them in a json file.
    returns number of suitable cases
    '''
    output_data = []
    for _ in range(iterations):
        B = randomMatrix(dimension, perimeter)
        v_min = shortestVector(B.LLL())
        lcLLL = B.solve_left(v_min)
        G = gram_matrix(B)
        this_case_works, rowindex = is_dysfunctional(B, v_min, lcLLL, G)
        if this_case_works !=functioning:
            case_info = into_dict(B, lcLLL, rowindex)
            output_data.append(case_info)
    if printing: print_listdict(output_data)
    dict = format_data(output_data)
    into_json(dict)
    return len(output_data)


def is_dysfunctional(B, v_min, lcLLL, G) -> bool:
    nonzero_ind = 0
    for current_row in range(dimension):
        lcCube = find_real_minimum(G, current_row, lcLLL[current_row])
        if lcCube == zero_vector(SR, dimension):
            continue
        nonzero_ind = current_row
        for i in range(dimension - 1):  
            difference = lcLLL[i] - lcCube[i]
            if abs(difference) >= sensitivity:
                return True, current_row
            else:
                if 1.003885269165039 == lcLLL[i]:
                    print(abs(difference))
    return False, nonzero_ind
        
def find_real_minimum(G, current_row, lcLLL) -> vector:
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([lcLLL * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1) * matrixB)).list()
    result.insert(current_row, lcLLL)
    return vector(result).n(digits=5)



In [95]:
def custom_serializer(obj):
    """Custom serializer for objects not serializable by default json code"""
    if hasattr(obj, 'tolist'):  # Checks if the object can be converted to a list
        return obj.tolist()
    elif hasattr(obj, '__iter__') and not isinstance(obj, str):  # Iterable but not a string
        return list(obj)
    elif isinstance(obj, (int, float)):  # Directly serializable numbers
        return float(obj)  # Ensure everything is serialized as a float for consistency
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")



# data_to_save = {
#     "B": B,
#     "G": G,
#     "lincomb_LLL": lincomb_LLL,
#     "lincomb_cube": lincomb_cube,
#     "sv_LLL": sv_LLL,
#     "sv_cube": sv_cube,
#     "lincomb_diff": lincomb_diff,
#     "LLL.norm": LLL_norm,
#     "cube.norm": cube_norm
# }

# into_json(data_to_save)


In [98]:

dimension = 3
perimeter = 50
sensitivity = 1
jsonfilename = "dummyfile.json"

generate_new_examples(100, dimension, perimeter, True, False)

[ 47 -30 -30]
[ 23 -17  37]
[-41  26  24] :  B
[ 4009   481 -3427]
[  481  2187  -497]
[-3427  -497  2933] :  G
(23, 1, 27) :  lincomb_LLL
(20.788, 1.0000, 24.459) :  lincomb_cube
(-3, -5, -5) :  sv_LLL
(-2.7725, -4.7129, 0.37012) :  sv_cube
(2.2120, 0.00000, 2.5412) :  lincomb_diff
7.6811 :  LLL.norm
5.4804 :  cube.norm



1